<a href="https://colab.research.google.com/github/Varat7v2/PERSON-DETECTION-TRAINING/blob/master/my_object_detection_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection API Demo
<table align="left"><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Import tensorflow with specific version

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

Go to tf-model/research/ folder

In [ ]:
%cd ~
%cd ..
%cd /content/drive/My Drive/Colab Notebooks/tf-model/research

### Run setup file

In [ ]:
!python setup.py build
!python setup.py install

#### Running setup file for /research/slim

In [ ]:
%cd slim
!python setup.py build
!python setup.py install
%cd ..

#### Compile protobufs and install the object_detection package

In [ ]:
!pwd
!protoc object_detection/protos/*.proto --python_out=.

#### Add Libraries to PYTHONPATH
/content/drive/My Drive/Colab Notebooks/tf-model/research

In [ ]:
!pwd
!export PYTHONPATH=$PYTHONPATH:"/content/drive/My Drive/Colab Notebooks/tf-model/research":"/content/drive/My Drive/Colab Notebooks/tf-model/research/slim"

#### Test that you have correctly installed the Tensorflow Object Detection API

In [ ]:
!python object_detection/builders/model_builder_test.py

### Training the model using OD API

In [ ]:
#  !python object_detection/legacy/train.py \
#  --clone_on_cpu=False --logtostderr \
#  --train_dir=/content/drive/My\ Drive/Colab\ Notebooks/ssdlite_checkpoints/ssd_mobilenet_v2_150x150_32f \
#  --pipeline_config_path=/content/drive/My\ Drive/Colab\ Notebooks/ssdlite_checkpoints/ssd_mobilenet_v2_150x150_32f/ssd_mobilenet_v2_150x150_32f_colab.config

Training --> PERSON DETECTION

In [ ]:
!python object_detection/legacy/train.py \
--logtostderr --clone_on_cpu=False \
--train_dir=/content/drive/My\ Drive/Colab\ Notebooks/person_detection_train/training/ckpt \
--pipeline_config_path=/content/drive/My\ Drive/Colab\ Notebooks/person_detection_train/training/person_detection_colab.config

### Running the tensorboard

In [ ]:
# !tensorboard --logdir /content/drive/My Drive/Colab Notebooks/ssdlite_checkpoints

### Imports

In [ ]:
# import numpy as np
# import os
# import six.moves.urllib as urllib
# import sys
# import tarfile
# import zipfile
# import os
# import pathlib

# from collections import defaultdict
# from io import StringIO
# from matplotlib import pyplot as plt
# from PIL import Image
# from IPython.display import display

Import the object detection module.

In [ ]:
# from object_detection.utils import ops as utils_ops
# from object_detection.utils import label_map_util
# from object_detection.utils import visualization_utils as vis_util

Patches:

In [ ]:
# # patch tf1 into `utils.ops`
# utils_ops.tf = tf.compat.v1

# # Patch the location of gfile
# tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [ ]:
# def load_model(model_name):
#   base_url = 'http://download.tensorflow.org/models/object_detection/'
#   model_file = model_name + '.tar.gz'
#   model_dir = tf.keras.utils.get_file(
#     fname=model_name, 
#     origin=base_url + model_file,
#     untar=True)

#   model_dir = pathlib.Path(model_dir)/"saved_model"

#   model = tf.saved_model.load(str(model_dir))
#   model = model.signatures['serving_default']

#   return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
# # List of the strings that is used to add correct label for each box.
# PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
# category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

For the sake of simplicity we will test on 2 images:

In [ ]:
# # If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
# TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
# TEST_IMAGE_PATHS

# Detection

Load an object detection model:

In [ ]:
# model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
# detection_model = load_model(model_name)

Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [ ]:
# print(detection_model.inputs)

And retuns several outputs:

In [ ]:
# detection_model.output_dtypes

In [ ]:
# detection_model.output_shapes

Add a wrapper function to call the model, and cleanup the outputs:

In [ ]:
# def run_inference_for_single_image(model, image):
#   image = np.asarray(image)
#   # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
#   input_tensor = tf.convert_to_tensor(image)
#   # The model expects a batch of images, so add an axis with `tf.newaxis`.
#   input_tensor = input_tensor[tf.newaxis,...]

#   # Run inference
#   output_dict = model(input_tensor)

#   # All outputs are batches tensors.
#   # Convert to numpy arrays, and take index [0] to remove the batch dimension.
#   # We're only interested in the first num_detections.
#   num_detections = int(output_dict.pop('num_detections'))
#   output_dict = {key:value[0, :num_detections].numpy() 
#                  for key,value in output_dict.items()}
#   output_dict['num_detections'] = num_detections

#   # detection_classes should be ints.
#   output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
#   # Handle models with masks:
#   if 'detection_masks' in output_dict:
#     # Reframe the the bbox mask to the image size.
#     detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
#               output_dict['detection_masks'], output_dict['detection_boxes'],
#                image.shape[0], image.shape[1])      
#     detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
#                                        tf.uint8)
#     output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
#   return output_dict

Run it on each test image and show the results:

In [ ]:
# def show_inference(model, image_path):
#   # the array based representation of the image will be used later in order to prepare the
#   # result image with boxes and labels on it.
#   image_np = np.array(Image.open(image_path))
#   # Actual detection.
#   output_dict = run_inference_for_single_image(model, image_np)
#   # Visualization of the results of a detection.
#   vis_util.visualize_boxes_and_labels_on_image_array(
#       image_np,
#       output_dict['detection_boxes'],
#       output_dict['detection_classes'],
#       output_dict['detection_scores'],
#       category_index,
#       instance_masks=output_dict.get('detection_masks_reframed', None),
#       use_normalized_coordinates=True,
#       line_thickness=4)

#   display(Image.fromarray(image_np))
#   return output_dict

In [ ]:
# for image_path in TEST_IMAGE_PATHS:
#   output = (show_inference(detection_model, image_path))

In [ ]:
# # idx = output['detection_scores'].argmax()
# # print(output['detection_scores'][idx])
# # output['detection_boxes'][idx]
# import matplotlib.patches as patches
# import tkinter
# num_objs = 0
# cords = []

# # print(output['detection_scores'])
# for i in range(output['detection_scores'].shape[0]):
#     if output['detection_scores'][i] > 0.6:
#         cords.append(output['detection_boxes'][i])
#         num_objs += 1
# print(num_objs)
# print(cords[0])
# print(cords[1])

# # # Create figure and axes
# # fig,ax = plt.subplots(1)
# # im = np.array(Image.open('models/research/object_detection/test_images/image3.jpg'), dtype=np.uint8)
# # # Display the image
# # ax.imshow(im)

# # # Create a Rectangle patch
# # rect = patches.Rectangle((cords[0][0]*255,cords[0][1]*255),height*255,width*255,linewidth=1,edgecolor='r',facecolor='none')
# # # Add the patch to the Axes
# # ax.add_patch(rect)

# # plt.savefig('foo.png')

# # # # plt.show()

## Instance Segmentation

In [ ]:
# model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
# masking_model = load_model("mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28")

The instance segmentation model includes a `detection_masks` output:

In [ ]:
# masking_model.output_shapes

In [ ]:
# for image_path in TEST_IMAGE_PATHS:
#   show_inference(masking_model, image_path)